# DataFrame's

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets

### Data Set

In [2]:
iris = datasets.load_iris()
df = pd.DataFrame(
    data=np.c_[iris['data'], iris['target']],
        columns= iris['feature_names'] + ['species'])

df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


### Select

In [3]:
# Coluna como vetor
# Não pode selecionar duas ou mais
# df["sepal length (cm)"] 

# Coluna como dataframe
df[ ["petal width (cm)", "species"] ]


,petal width (cm),species
0,0.2,0.0
1,0.2,0.0
2,0.2,0.0
3,0.2,0.0
4,0.2,0.0
...,...,...
145,2.3,2.0
146,1.9,2.0
147,2.0,2.0
148,2.3,2.0


### Tranformação de uma coluna

In [4]:
species = {0 : "setosa", 1: "versicolor", 2: "virginica"}
df["species"] = df["species"].apply(lambda x: species[x])
original = df.copy()
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


### Mutate

In [5]:
df["sepal_area"] = df["sepal length (cm)"] * df["sepal width (cm)"]
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,sepal_area
0,5.1,3.5,1.4,0.2,setosa,17.85
1,4.9,3.0,1.4,0.2,setosa,14.70
2,4.7,3.2,1.3,0.2,setosa,15.04
3,4.6,3.1,1.5,0.2,setosa,14.26
4,5.0,3.6,1.4,0.2,setosa,18.00


### Filter

In [6]:
# df.loc[df["species"] == "setosa"]
df.query("species == 'setosa'", inplace=True)
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,sepal_area
45,4.8,3.0,1.4,0.3,setosa,14.40
46,5.1,3.8,1.6,0.2,setosa,19.38
47,4.6,3.2,1.4,0.2,setosa,14.72
48,5.3,3.7,1.5,0.2,setosa,19.61
49,5.0,3.3,1.4,0.2,setosa,16.50


### Group by + Count

In [7]:
original.groupby("species").size().reset_index(name='n')

,species,n
0,setosa,50
1,versicolor,50
2,virginica,50


In [8]:
original.groupby("species").mean()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [9]:
(original
    .loc[:, ["sepal length (cm)", "species"]]
    .query("`sepal length (cm)` < 6.0")
    .groupby("species")
    .median()
    .reset_index()
)

,species,sepal length (cm)
0,setosa,5.0
1,versicolor,5.6
2,virginica,5.8


# Pyspark

In [5]:
from pyspark.sql import SparkSession, Row

spark = SparkSession.builder.getOrCreate()

### Leitura do banco

In [17]:
df = spark.read.csv("../Listas/Lista_1/dados/AC-Parte_1.csv", header=True, sep=";")
df.createOrReplaceTempView("t1")

### Querys em SQL

In [18]:
seg_dose = f"""
    SELECT * FROM t1
    WHERE (
        vacina_descricao_dose='2ª Dose' OR
        vacina_descricao_dose='2ª Dose Revacinação')
    LIMIT 1
"""

fast_join = f"""
    SELECT name_health_region as regiao_saude
    FROM ({seg_dose}) as seg_dose
    LEFT JOIN codigos
    ON seg_dose.estabelecimento_municipio_codigo
    = codigos.est_mun_codigo
"""

qnt_vax = f"""
    SELECT regiao_saude, COUNT(*) AS N
    FROM ({fast_join})
    GROUP BY regiao_saude
"""

faixa = f"""
    WITH qntVax AS ({qnt_vax})
    SELECT regiao_saude, N,
        CASE WHEN N > (
            SELECT percentile_approx(N, 0.5) 
            FROM qntVax)
        THEN 'Alto'
        ELSE 'Baixo'
        END AS Faixa
        FROM qntVax
"""

bot5 = f"""
    WITH tabFaixa AS ({faixa})
    SELECT regiao_saude, N, Faixa 
    FROM (
        SELECT *, dense_rank()
        OVER (PARTITION BY Faixa ORDER BY N DESC) as posicao
        FROM tabFaixa
    )
    WHERE posicao <= 5
"""

### Extração dos resultados

In [22]:
spark.sql(fast_join).show()

AnalysisException: Table or view not found: codigos; line 10 pos 14;
'Project ['name_health_region AS regiao_saude#519]
+- 'Join LeftOuter, ('seg_dose.estabelecimento_municipio_codigo = 'codigos.est_mun_codigo)
   :- SubqueryAlias seg_dose
   :  +- GlobalLimit 1
   :     +- LocalLimit 1
   :        +- Project [document_id#61, paciente_id#62, paciente_idade#63, paciente_dataNascimento#64, paciente_enumSexoBiologico#65, paciente_racaCor_codigo#66, paciente_racaCor_valor#67, paciente_endereco_coIbgeMunicipio#68, paciente_endereco_coPais#69, paciente_endereco_nmMunicipio#70, paciente_endereco_nmPais#71, paciente_endereco_uf#72, paciente_endereco_cep#73, paciente_nacionalidade_enumNacionalidade#74, estabelecimento_valor#75, estabelecimento_razaoSocial#76, estalecimento_noFantasia#77, estabelecimento_municipio_codigo#78, estabelecimento_municipio_nome#79, estabelecimento_uf#80, vacina_grupoAtendimento_codigo#81, vacina_grupoAtendimento_nome#82, vacina_categoria_codigo#83, vacina_categoria_no

# Pandas

In [2]:
from datetime import datetime, date
import pandas as pd

In [3]:
df = pd.read_csv("../Listas/Lista_1/dados/AC-Parte_1.csv", sep=";")
df.head()


c:\Users\Jose\Desktop\Topicos\Trabalho\venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,document_id,paciente_id,paciente_idade,paciente_dataNascimento,paciente_enumSexoBiologico,paciente_racaCor_codigo,paciente_racaCor_valor,paciente_endereco_coIbgeMunicipio,paciente_endereco_coPais,paciente_endereco_nmMunicipio,...,vacina_categoria_codigo,vacina_categoria_nome,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataAplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem
0,1a44aaf2-0dd6-4289-809e-cb8de255e60d-i0b0,5266947cd5cd2f366f58be8aaad7d380eb9ab63ced4151...,22.0,2000-02-24,M,99.0,SEM INFORMACAO,120020.0,10.0,CRUZEIRO DO SUL,...,2.0,Faixa Etária,FF8848,PFIZER,Organization/28290,2022-03-24,2ª Dose,87,COVID-19 PFIZER - COMIRNATY,Novo PNI
1,9837350c-d21a-4e0a-b210-5e34fbde63c3-i0b0,e3b09d2581d3f53160e536422f599e5b2f220012122a0d...,33.0,1987-06-29,F,3.0,PARDA,120042.0,10.0,RODRIGUES ALVES,...,9.0,Trabalhadores de Saúde,4120Z004,ASTRAZENECA/FIOCRUZ,Organization/33781055000135,2021-02-17,1ª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,Novo PNI
2,4e15de2c-8226-4208-a510-a49398d202e9-i0b0,1cdfdf4bc68e515dcc74fe99bcbc7ee26eef857164df11...,15.0,2006-06-22,F,99.0,SEM INFORMACAO,120050.0,10.0,SENA MADUREIRA,...,2.0,Faixa Etária,FA9096,PFIZER,Organization/00394544000851,2021-08-11,1ª Dose,87,COVID-19 PFIZER - COMIRNATY,Novo PNI
3,983d9d5f-456d-4249-84fe-8b58f3d855c6-i0b0,f07d519e6b587e3709147a6c096cc1a3ba835d4f7417f9...,50.0,1970-10-15,M,1.0,BRANCA,120060.0,10.0,TARAUACA,...,2.0,Faixa Etária,217VCD239Z,ASTRAZENECA/FIOCRUZ,Organization/33781055000135,2021-09-29,2ª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,Novo PNI
4,75e6901f-cee7-45e7-a728-7138f5e88857-i0b0,7b8385b8818eff68cd97ddf978f053e0ffba168914838e...,18.0,2003-08-23,F,4.0,AMARELA,120038.0,10.0,PLACIDO DE CASTRO,...,114.0,Outros,FD7209,PFIZER,Organization/00394544000851,2021-09-30,2ª Dose,87,COVID-19 PFIZER - COMIRNATY,Novo PNI
